In [1]:
from transformers import pipeline
import librosa
import time
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

nums = []
# Load the audio classification pipeline with your chosen model
def process_model(model_name):
    try:
        pipe = pipeline("audio-classification", model=model_name)
    except:
        print("This model doesn't have a pipeline.")
# Load an audio file
    array, sampling_rate = librosa.load("audio-samples/table-smack.mp3", sr=16000)  # Ensure correct sample rate

# Measure inference time
    for x in range(1,6):
        start_time = time.time()
        predictions = pipe(array, top_k=5)  
        inference_time = time.time() - start_time
        nums.append(inference_time)

    print(f"Inference time: {inference_time} seconds")
    print(pipe({"path" : "" , "array" : array, "sampling_rate" : sampling_rate}))

    nums_sum = sum(nums)
    nums_length = len(nums)
    inference_time = nums_sum/nums_length
    print(nums)
    print(inference_time)
    
    return predictions, inference_time

file_path = 'model_results.xlsx'

if os.path.exists(file_path):
    results_df = pd.read_excel(file_path)
else:
    results_df = pd.DataFrame(columns=['Model Name', 'Inference Time', 'Predictions'])


def update_or_append(results_df, model_name, inference_time, predictions):
    if model_name in results_df['Model Name'].values:
        return
    else:
        new_row = {
            'Model Name': model_name,
            'Inference Time': inference_time,
            'Predictions': predictions
        }
        return new_row
    
with open('model_names.txt', 'r') as file:
    for model_name in file:
        model_name = model_name.strip()
        inference_time, predictions = process_model(model_name)
        try:
            new_row_df = pd.DataFrame([update_or_append(results_df, model_name, inference_time, predictions)])
            results_df = pd.concat([results_df, new_row_df], ignore_index=True)
        except:
            print("You already have the same model in the excel.")
            
        results_df.to_excel(file_path, index=False)

# Load the workbook and select the active worksheet
wb = load_workbook(file_path)
ws = wb.active

# Auto-fit column widths
for col in ws.columns:
    max_length = 0
    column = col[0].column  # Get the column name
    for cell in col:
        try:  # Necessary to avoid error on empty cells
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    ws.column_dimensions[get_column_letter(column)].width = adjusted_width

# Save the workbook
wb.save(file_path)

fastest_time = results_df['Inference Time'].min()

fastest_model_names = results_df.loc[results_df['Inference Time'] == fastest_time, 'Model Name']
fastest_model_name = fastest_model_names.iloc[0]
print(f"The fastest model is: {fastest_model_name} with the time being: {fastest_time}")

: 